In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime, timedelta

### Quote Data preparation for training Hidden Markov Models on High-Frequency Quote Data

- Data preprocessing and cleaning
- Parsing observation vectors from raw data
- Labelling hidden states within training dataset

In [4]:
quotes=pd.read_csv('data/cleaned_quotes.csv',index_col=0)


simple_quotes=quotes[['Exchange','Symbol','Bid_Price','Bid_Size','Offer_Price','Offer_Size']].copy()
simple_quotes=simple_quotes[simple_quotes.index<"2020-01-03"]

simple_quotes['OB_IB']=simple_quotes['Offer_Size']/simple_quotes['Bid_Size']
simple_quotes['spread']=simple_quotes['Offer_Price']-simple_quotes['Bid_Price']
simple_quotes

C:\Users\jbohn\AppData\Local\Temp\ipykernel_2680\1377133329.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  quotes=pd.read_csv('data/cleaned_quotes.csv',index_col=0)


,Exchange,Symbol,Bid_Price,Bid_Size,Offer_Price,Offer_Size,OB_IB,spread
2020-01-02 09:30:00.134062,P,AAPL,296.09,1.0,296.29,1.0,1.000000,0.20
2020-01-02 09:30:00.134336,K,AAPL,296.21,1.0,296.39,1.0,1.000000,0.18
2020-01-02 09:30:00.134532,K,AAPL,296.10,1.0,296.36,2.0,2.000000,0.26
2020-01-02 09:30:00.136081,K,AAPL,296.10,1.0,296.29,1.0,1.000000,0.19
2020-01-02 09:30:00.234474,K,AAPL,296.11,1.0,296.39,1.0,1.000000,0.28
...,...,...,...,...,...,...,...,...
2020-01-02 19:59:43.952528,P,AAPL,300.79,2.0,301.34,1.0,0.500000,0.55
2020-01-02 19:59:52.610437,P,AAPL,300.79,2.0,301.19,1.0,0.500000,0.40
2020-01-02 19:59:52.611152,P,AAPL,300.79,2.0,301.34,1.0,0.500000,0.55
2020-01-02 19:59:52.611817,Q,AAPL,301.19,1.0,302.00,6.0,6.000000,0.81


### Label our States

- Spread Tightens or Widens compared to last BBO
- Top of Book Relation to last BBO

In [5]:

simple_quotes['last_spread']=simple_quotes['spread'].shift(1)

#label state as 1 or -1 based on widen or tighten
def label_spread(x):
    if x['spread']>x['last_spread']:
        # spread widens
        return 1
    elif x['spread']<x['last_spread']:
        # spread tightens 
        return -1
    else:
        return 0
simple_quotes['spread_state']=simple_quotes.apply(lambda x:label_spread(x),axis=1)
simple_quotes=simple_quotes.drop(columns=['last_spread'])



In [6]:
simple_quotes

,Exchange,Symbol,Bid_Price,Bid_Size,Offer_Price,Offer_Size,OB_IB,spread,spread_state
2020-01-02 09:30:00.134062,P,AAPL,296.09,1.0,296.29,1.0,1.000000,0.20,0
2020-01-02 09:30:00.134336,K,AAPL,296.21,1.0,296.39,1.0,1.000000,0.18,-1
2020-01-02 09:30:00.134532,K,AAPL,296.10,1.0,296.36,2.0,2.000000,0.26,1
2020-01-02 09:30:00.136081,K,AAPL,296.10,1.0,296.29,1.0,1.000000,0.19,-1
2020-01-02 09:30:00.234474,K,AAPL,296.11,1.0,296.39,1.0,1.000000,0.28,1
...,...,...,...,...,...,...,...,...,...
2020-01-02 19:59:43.952528,P,AAPL,300.79,2.0,301.34,1.0,0.500000,0.55,1
2020-01-02 19:59:52.610437,P,AAPL,300.79,2.0,301.19,1.0,0.500000,0.40,-1
2020-01-02 19:59:52.611152,P,AAPL,300.79,2.0,301.34,1.0,0.500000,0.55,1
2020-01-02 19:59:52.611817,Q,AAPL,301.19,1.0,302.00,6.0,6.000000,0.81,1


### Analysis of Features

- Observations are a discrete continuous vector within R^4
- Hidden States are binary (1, -1)

In [7]:
features=simple_quotes[['Bid_Size', 'Offer_Size', 'spread','OB_IB']].copy()
features.to_csv('data/features.csv')
features


,Bid_Size,Offer_Size,spread,OB_IB
2020-01-02 09:30:00.134062,1.0,1.0,0.20,1.000000
2020-01-02 09:30:00.134336,1.0,1.0,0.18,1.000000
2020-01-02 09:30:00.134532,1.0,2.0,0.26,2.000000
2020-01-02 09:30:00.136081,1.0,1.0,0.19,1.000000
2020-01-02 09:30:00.234474,1.0,1.0,0.28,1.000000
...,...,...,...,...
2020-01-02 19:59:43.952528,2.0,1.0,0.55,0.500000
2020-01-02 19:59:52.610437,2.0,1.0,0.40,0.500000
2020-01-02 19:59:52.611152,2.0,1.0,0.55,0.500000
2020-01-02 19:59:52.611817,1.0,6.0,0.81,6.000000


In [8]:
outcomes=simple_quotes[['spread_state']].copy()
outcomes.to_csv('data/outcomes.csv')
outcomes

,spread_state
2020-01-02 09:30:00.134062,0
2020-01-02 09:30:00.134336,-1
2020-01-02 09:30:00.134532,1
2020-01-02 09:30:00.136081,-1
2020-01-02 09:30:00.234474,1
...,...
2020-01-02 19:59:43.952528,1
2020-01-02 19:59:52.610437,-1
2020-01-02 19:59:52.611152,1
2020-01-02 19:59:52.611817,1
